In [54]:
import torch
import torch.nn.functional as F
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn
from torch.utils.data import DataLoader  
from torchvision import datasets
import torchvision.transforms as transforms  # 


In [55]:
from Chess_Data import ChessData

In [56]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [72]:
class Test_Model(nn.Module):
    def __init__(self):
        super(Test_Model,self).__init__()
        self.conv1 = nn.Conv2d(7,128, kernel_size=3,stride=1)
        self.max = nn.MaxPool2d(2,2)
        self.linear = nn.Sequential(nn.Linear(9*128, 512), nn.ReLU(), nn.Linear(512,512), nn.ReLU(), nn.Linear(512,128),
                                   nn.ReLU(), nn.Linear(128,64), nn.ReLU(), nn.Linear(64,2))
        

    def init_weights(self):
        torch.nn.init.normal_(self.conv1.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv2.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv3.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv4.weight,0, 1e-6)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.max(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x)
        
        
        return x


        

In [73]:
trainset =  ChessData('training.csv')
trainloader= torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0 )

In [74]:
def train(model,criterion,optimizer,num_epochs,trainloader):
    
    for epoch in range(num_epochs):
        running_loss=0.0
        
        for i , (  targets, data) in enumerate(trainloader):
            
            data = data.to(device)
            targets= targets.to(device)
            targets = targets.to(torch.int64)

            #forward prop
            scores = model(data).squeeze()
            

            loss = criterion(scores, targets)

            #backprop
            optimizer.zero_grad()
            loss.backward()

            #take step
            optimizer.step()
            losss= loss.item()
            
            running_loss+=losss
            
            
           
            if i %500==499:
                print('[%d,%5d] loss: %.3f' % (epoch+1 , i+1, running_loss/500))
                running_loss=0.0
            if i % 2000 == 1999: 
                print("Train accuracy: "+ str(check_accuracy(trainloader, model)*100))
               # print("Test accuracy: "+ str(check_accuracy(testloader, model)*100))
            

In [60]:
def check_accuracy(loader, model, full=False):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for  i ,( y, x) in enumerate(loader):
            if i >100 and not full:

                return num_correct/num_samples
            x = x.to(device)
            y = y.to(device)
            y =y.to(torch.int64)
           

            scores = model(x)
            
            
            _, predictions = scores.max(1)
            
            num_correct += (predictions == y).sum()
            
         
            
            
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

In [61]:
my_model = Test_Model().to(device)

In [ ]:


batch_size = 8
learning_rate = .00000001
num_epochs = 10
criterion3 = nn.CrossEntropyLoss()
#optimizer3=optim.Adam(my_model.parameters(),lr=learning_rate)
optimizer3 = optim.SGD(my_model.parameters(),lr=learning_rate,momentum=.9)
num_epochs=15
train(my_model,criterion3,optimizer3,num_epochs,trainloader)

#check_accuracy(trainloader, my_model)

[1,  500] loss: 0.502
[1, 1000] loss: 0.498
[1, 1500] loss: 0.497
[1, 2000] loss: 0.492
Train accuracy: tensor(75.1238, device='cuda:0')
[1, 2500] loss: 0.512
[1, 3000] loss: 0.493
[1, 3500] loss: 0.498
[1, 4000] loss: 0.501
Train accuracy: tensor(74.0718, device='cuda:0')
[1, 4500] loss: 0.500
[1, 5000] loss: 0.496
[1, 5500] loss: 0.501
[1, 6000] loss: 0.497
Train accuracy: tensor(73.1436, device='cuda:0')
[1, 6500] loss: 0.506
[1, 7000] loss: 0.483
[1, 7500] loss: 0.491
[1, 8000] loss: 0.499
Train accuracy: tensor(76.3614, device='cuda:0')
[1, 8500] loss: 0.513
[1, 9000] loss: 0.488
[1, 9500] loss: 0.502
[1,10000] loss: 0.502
Train accuracy: tensor(73.7624, device='cuda:0')
[1,10500] loss: 0.488
[1,11000] loss: 0.504
[1,11500] loss: 0.498
[1,12000] loss: 0.498
Train accuracy: tensor(75.4950, device='cuda:0')
[1,12500] loss: 0.487
[1,13000] loss: 0.494
[1,13500] loss: 0.496
[1,14000] loss: 0.502
Train accuracy: tensor(75.6807, device='cuda:0')
[1,14500] loss: 0.496
[1,15000] loss: 0.4

Train accuracy: tensor(76.7327, device='cuda:0')
[1,120500] loss: 0.497
[1,121000] loss: 0.508
[1,121500] loss: 0.509
[1,122000] loss: 0.496
Train accuracy: tensor(72.7723, device='cuda:0')
[1,122500] loss: 0.490
[1,123000] loss: 0.505
[1,123500] loss: 0.502
[1,124000] loss: 0.500
Train accuracy: tensor(75.2475, device='cuda:0')
[1,124500] loss: 0.493
[1,125000] loss: 0.507
[1,125500] loss: 0.499
[1,126000] loss: 0.488
Train accuracy: tensor(74.8144, device='cuda:0')
[1,126500] loss: 0.494
[1,127000] loss: 0.497
[1,127500] loss: 0.490
[1,128000] loss: 0.493
Train accuracy: tensor(75.9901, device='cuda:0')
[1,128500] loss: 0.498
[1,129000] loss: 0.495
[1,129500] loss: 0.500
[1,130000] loss: 0.505
Train accuracy: tensor(75.6188, device='cuda:0')
[1,130500] loss: 0.500
[1,131000] loss: 0.485
[1,131500] loss: 0.495
[1,132000] loss: 0.503
Train accuracy: tensor(73.3292, device='cuda:0')
[1,132500] loss: 0.499
[1,133000] loss: 0.486
[1,133500] loss: 0.484
[1,134000] loss: 0.495
Train accurac

[1,236500] loss: 0.503
[1,237000] loss: 0.496
[1,237500] loss: 0.504
[1,238000] loss: 0.504
Train accuracy: tensor(74.8144, device='cuda:0')
[1,238500] loss: 0.489
[1,239000] loss: 0.497
[1,239500] loss: 0.506
[1,240000] loss: 0.502
Train accuracy: tensor(75.5569, device='cuda:0')
[1,240500] loss: 0.512
[1,241000] loss: 0.499
[1,241500] loss: 0.494
[1,242000] loss: 0.505
Train accuracy: tensor(75.6188, device='cuda:0')
[1,242500] loss: 0.508
[1,243000] loss: 0.495
[1,243500] loss: 0.498
[1,244000] loss: 0.509
Train accuracy: tensor(74.1337, device='cuda:0')
[1,244500] loss: 0.501
[1,245000] loss: 0.493
[1,245500] loss: 0.491
[1,246000] loss: 0.495
Train accuracy: tensor(74.8762, device='cuda:0')
[1,246500] loss: 0.508
[1,247000] loss: 0.504
[1,247500] loss: 0.503
[1,248000] loss: 0.503
Train accuracy: tensor(74.3193, device='cuda:0')
[1,248500] loss: 0.494
[1,249000] loss: 0.496
[1,249500] loss: 0.496
[1,250000] loss: 0.490
Train accuracy: tensor(73.9480, device='cuda:0')
[1,250500] lo

In [71]:
torch.save(my_model.state_dict(), './Prototype2.0-Adam-SGD-1C5L0B-2.pth')


In [ ]:
test_model = Test_Model().to(device)
test_model.load_state_dict(torch.load('./Prototype1.0-Adam/SGD-2C3L0B-1.pth'))
check_accuracy(trainloader, test_model, full=True)

In [ ]:
'''./Prototype2.0-Adam-SGD-1C5L0B-2.pth
class Test_Model(nn.Module):
    def __init__(self):
        super(Test_Model,self).__init__()
        self.conv1 = nn.Conv2d(7,128, kernel_size=3,stride=1)
        self.max = nn.MaxPool2d(2,2)
        self.linear = nn.Sequential(nn.Linear(1152, 512), nn.ReLU(), nn.Linear(512,512), nn.ReLU(), nn.Linear(512,128),
                                   nn.ReLU(), nn.Linear(128,64), nn.ReLU(), nn.Linear(64,2))
        

    def init_weights(self):
        torch.nn.init.normal_(self.conv1.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv2.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv3.weight,0, 1e-7)
        torch.nn.init.normal_(self.conv4.weight,0, 1e-6)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.max(x)
        x = x.reshape(x.shape[0], -1)
        print(x.shape)
        x = self.linear(x)
        
        
        return x
'''
